In [1]:
import all_functions as af

Importing add_TimeWindowsCapacity
Importing checkCapacityInsertPts


In [2]:
def busReschedule_run(schedule_filename,
                      accesskey,
                      secretkey,
                      bookingid,
                      broken_run,
                      windows = 1800.,
                      resched_init_time = None,
                      path_to_outdir = af.os.path.join(af.os.getcwd(),'data'),
                      radius = 3.):
    
    flag = 400 #400 is good, 200 is bad.

    if not af.os.path.exists(path_to_outdir):
        path_to_outdir = af.os.join.path(af.os.getcwd(),'data')

    #get data
    if schedule_filename is not None:
        if af.os.path.isfile(schedule_filename):
            fullSchedule = af.pd.DataFrame.from_csv(schedule_filename, header=0, sep=',', index_col = False)
        else:
            print('File not found!')
            flag = 200
            return flag

    if accesskey and secretkey is not None:
        #AWS_ACCESS_KEY = raw_input("Please enter AWS access key: ")
        #AWS_SECRET_KEY = raw_input("Please enter AWS secret key: ")
        try:
            fullSchedule = af.s3_data_acquire(accesskey, secretkey, path_to_outdir, qc_file_name = 'qc_streaming.csv')
            if type(fullSchedule) == int:
                flag = 200
                return flag

        except IOError: #is this the right error if s3_data_acquire fails?
            print('Could not access streaming data!')
            flag = 200
            return flag
            
    #Determine broken run number, or get list of unhandled requests.
    if broken_run is not None:
        case = 'BROKEN_RUN'
    elif bookingid is not None:
        case = 'INDIVIDUAL_REQUESTS'
        individual_requests = list(bookingid)

    # this simply returns full schedule with time windows at the moment
    sched_obj = af.aTWC.TimeWindowsCapacity(fullSchedule)
    fullSchedule_windows = sched_obj.addtoRun_TimeCapacity(windows)
    fS_w_copy = fullSchedule_windows.copy()

    #this gets us all the URIDs for the broken run given the initial rescheduling time
    #OR it will get us URIDs given specific bookingIds to be rescheduled
    if case == 'BROKEN_RUN':
        if broken_run not in list(set(fullSchedule_windows.Run.tolist())):
            print('Run number is not scheduled for today!')
            flag = 200
            return flag

        if resched_init_time is not None:
            resched_init_time = af.humanToSeconds(resched_init_time)

        if resched_init_time is None:
            t = af.datetime.datetime.now()
            hr = str(t.hour)
            if t.minute < 10:
                m = str(0)+str(t.minute)
            else:
                m = str(t.minute)
            t = hr+':'+m
            resched_init_time = af.humanToSeconds(t)

        URIDs = af.get_URID_Bus(fullSchedule_windows, broken_run, resched_init_time) 

    else:
        for i in range(len(individual_requests)):
            if individual_requests[i] not in list(set(fullSchedule_windows.BookingId.tolist())):
                print('You have entered BookingIds not present in the schedule!')
                flag = 200
                return flag

        URIDs = af.get_URID_BookingIds(fullSchedule_windows, individual_requests)

    if not URIDs:
        print('There are no people to reschedule on bus {0} at time {1}'.format(broken_run, resched_init_time))
        flag = 200
        return flag

    # for each URID we find the bus runs to check through a radius elimination.
    # for each URID for each run we then want to check the capacity in the given time
    # window and return the URID with updated insert points. This URID with updated
    # insert points is fed to the feasibilty function, which we ultimately want to return
    # a minimum cost run for the URID and that run updated with the new URID slotted in.
    taxi_costs = []
    delay_costs = []
    best_buses = []
    for i in range(len(URIDs)):
        print('Rescheduling URID {0}'.format(i))
        busRuns_tocheck = af.radius_Elimination(fullSchedule_windows, URIDs[i], radius=radius)
        insert_stats = []

        #iterate over all runs, find best one!
        for run in busRuns_tocheck:

            this_run = fullSchedule_windows[fullSchedule_windows['Run']==run]
            capacity_obj = af.checkCap.CapacityInsertPts(this_run)

            #Kristen's capacity checker:
            URIDs[i].PickupInsert, URIDs[i].DropoffInsert = capacity_obj.return_inserts(URIDs[i])

            # IF THERE'S ROOM: TEST FEASIBILITY
            if not af.np.isnan(URIDs[i].PickupInsert):
                URIDs[i].PickupStart = URIDs[i].PickupInsert
                URIDs[i].DropoffStart = URIDs[i].DropoffInsert

                runSchedule = af.get_busRuns(fullSchedule_windows, run, None)

                print('Testing feasibility for run ' + run)
                brokenwindows_dict =af.insertFeasibility(runSchedule, URIDs[i])
                if not brokenwindows_dict:
                    print('Run {0} infeasible without moving the return-to-garage row.'.format(run))
                else:
                    insert_stats.append(brokenwindows_dict)

        #ASSEMBLE and ORDER transit options.
        if insert_stats:
            #ORDER buses by lowest additional lag time, i.e. total_lag, and sequentially add total_lag's
            ordered_inserts = sorted(insert_stats, key = af.operator.itemgetter('additional_time'))
        
            popme = []
            for k in range(len(ordered_inserts)):
                if ordered_inserts[k]['RunID']==broken_run:
                    popme.append(k)
            if popme:
                ordered_inserts.pop(popme)


            delay_costs.append(ordered_inserts[0]['additional_time'][0]*(48.09/3600)) #total dollars
            best_buses.append(ordered_inserts[0]['RunID'])

            #CALCULATE taxi cost
            taxi_costs.append(af.taxi(URIDs[i].PickUpCoords[0], URIDs[i].PickUpCoords[1],
                URIDs[i].DropOffCoords[0], URIDs[i].DropOffCoords[1], af.wheelchair_present(URIDs[i])))

            #WRITE information about best insertions to text file
            af.write_insert_data(URIDs[i], ordered_inserts[0:3],
                path_to_outdir, taxi_costs[i])

            #UPDATE whole day's schedule:
            fullSchedule_windows = af.day_schedule_Update(data = fullSchedule_windows, top_Feasibility = ordered_inserts[0], URID = URIDs[i])
            sched_obj_update = af.aTWC.TimeWindowsCapacity(fullSchedule_windows)
            fullSchedule_windows = sched_obj_update.add_Capacity(update = True)

            #SAVE just the updated run for each URID
            fullSchedule_windows[fullSchedule_windows['Run'] == ordered_inserts[0]['RunID']].to_csv(af.os.path.join(path_to_outdir, str(str(int(URIDs[i].BookingId))+'_schedule.csv')), index = False)

        else:
            delay_costs.append(400000)
            taxi_costs.append(af.taxi(URIDs[i].PickUpCoords[0], URIDs[i].PickUpCoords[1],
                URIDs[i].DropOffCoords[0], URIDs[i].DropOffCoords[1], af.wheelchair_present(URIDs[i])))
            best_buses.append('NA')

    #WRITE csv of PREFERRED OPTIONS:
    if case == 'BROKEN_RUN':
        nrun_cost = af.newBusRun_cost(af.get_busRuns(fS_w_copy, broken_run, URIDs[0]), provider = 6)
    else:
        nrun_cost = None
    pref_opt = af.preferred_options(URIDs, best_buses, delay_costs, taxi_costs, nrun_cost)
    pref_opt.to_csv(af.os.path.join(path_to_outdir, 'preferred_options.csv'), index = False)

    return flag

In [3]:
busReschedule_run(schedule_filename = '/Users/fineiskid/Desktop/DSSG_ffineis/main_repo/Access_Analysis_Rproject/data/output/qc_streaming.csv', 
                  accesskey = '' , 
                  secretkey = '', 
                  bookingid = None, 
                  broken_run = '6020', 
                  windows = 1800.,
                  resched_init_time = '11:30', 
                  path_to_outdir = '/Users/fineiskid/Desktop/DSSG_ffineis/main_repo/Access_Analysis_Rproject/data/output', 
                  radius = 3.)
    

There are 2 rides left to be scheduled on broken run 6020
Rescheduling URID 0
DropoffStart before PickupEnd
entire window available!
Testing feasibility for run 5063
    break_TW  late  nodes
0          0     0   6580
1          0     0   6581
2          0     0   6582
0          0     0   6583
1          0     0   6584
2          0     0   6585
3          0     0   6586
4          1    55   6587
5          1   981   6588
6          0     0   6589
7          0     0   6590
8          0     0   6591
9          0     0   6592
10         0     0   6593
11         1   972   6594
12         0     0   6595
13         0     0   6596
14         0     0   6597
0
DropoffStart after PickupEnd
returning pick up and drop off inserts
Testing feasibility for run 4053
   break_TW  late  nodes
0         0     0   2314
1         0     0   2315
2         0     0   2316
0         0     0   2317
1         1   481   2318
2         1   173   2319
3         1   830   2320
4         1    83   2321
5         1 

/Users/fineiskid/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/fineiskid/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s



1
Rescheduling URID 1
DropoffStart before PickupEnd
entire window available!
Testing feasibility for run 4053
   break_TW  late  nodes
0         0     0   2312
1         0     0   2313
2         0     0   2314
0         0     0   2315
1         1   168   2316
2         0     0   2317
3         1   517   2318
4         0     0   2319
5         1  1444   2320
6         1  1146   2321
7         0     0   2322
8         0     0   2323
3
DropoffStart after PickupEnd
returning pick up and drop off inserts
Testing feasibility for run 4052
    break_TW  late  nodes
0          0     0   2288
1          0     0   2289
2          0     0   2290
3          0     0   2291
0          1   724   2292
1          0     0   2293
2          1   974   2294
3          1    99   2295
4          0     0   2296
5          0     0   2297
6          1   559   2298
7          0     0   2299
8          0     0   2300
9          0     0   2301
10         0     0   2302
11         0     0   2303
12         0     0 

400